In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch

In [46]:
N = 1000
MAX_SEQ_LEN = 10
MAX_INT = 8

# Memorize first
# X = torch.randint(1, MAX_INT + 1, (N,MAX_SEQ_LEN)).long()
# Y = X[:,0]
# vocab_size = MAX_INT + 1

# Notice pre-marker
X = []
for i in range(N):
    seq_len = int(torch.randint(3, MAX_SEQ_LEN + 1, (1,)))
    X.append(torch.randint(1, MAX_INT + 1, (seq_len,)).long())
Y = torch.zeros(N).long()
for i in range(N):
    needle = np.random.randint(2, len(X[i]))
    X[i][needle] = MAX_INT + 1
    Y[i] = X[i][needle - 1]
vocab_size = MAX_INT + 2

from torch.nn.utils.rnn import pad_sequence
X2 = sorted(X, key=lambda x:-len(x))
X3 = pad_sequence(X2, batch_first=True)

# input_size = 4
# hidden_size = 10
# bidirectional = False

# Xs = [X[:800], X[800:900], X[900:]]
# Ys = [Y[:800], Y[800:900], Y[900:]]

In [47]:
X3

tensor([[ 4,  3,  1,  ...,  7,  9,  8],
        [ 1,  8,  4,  ...,  1,  6,  8],
        [ 3,  3,  5,  ...,  7,  9,  2],
        ...,
        [ 2,  5,  9,  ...,  0,  0,  0],
        [ 5,  3,  9,  ...,  0,  0,  0],
        [ 1,  7,  9,  ...,  0,  0,  0]])

In [53]:
X3.shape

torch.Size([1000, 10])

In [57]:
X = X3[np.random.randint(0,1000,32),:]
X.shape
X

torch.Size([32, 10])

tensor([[ 1,  2,  8,  7,  9,  4,  3,  5,  1,  5],
        [ 5,  8,  3,  9,  0,  0,  0,  0,  0,  0],
        [ 6,  6,  9,  1,  8,  2,  3,  8,  0,  0],
        [ 1,  8,  9,  6,  3,  0,  0,  0,  0,  0],
        [ 2,  3,  9,  0,  0,  0,  0,  0,  0,  0],
        [ 2,  7,  9,  0,  0,  0,  0,  0,  0,  0],
        [ 5,  6,  7,  6,  9,  2,  7,  0,  0,  0],
        [ 1,  4,  1,  7,  8,  1,  9,  0,  0,  0],
        [ 5,  6,  2,  6,  2,  9,  7,  1,  1,  4],
        [ 6,  2,  8,  4,  9,  1,  8,  4,  4,  0],
        [ 3,  5,  1,  9,  5,  5,  0,  0,  0,  0],
        [ 6,  5,  9,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  5,  9,  0,  0,  0,  0,  0,  0,  0],
        [ 4,  6,  9,  7,  2,  1,  0,  0,  0,  0],
        [ 2,  7,  9,  0,  0,  0,  0,  0,  0,  0],
        [ 5,  6,  9,  0,  0,  0,  0,  0,  0,  0],
        [ 7,  1,  7,  9,  5,  7,  5,  7,  0,  0],
        [ 6,  6,  9,  1,  5,  3,  6,  4,  0,  0],
        [ 4,  6,  2,  2,  1,  2,  1,  9,  1,  0],
        [ 1,  7,  9,  6,  0,  0,  0,  0,  0,  0],


In [58]:
batch_size, max_seq = X.shape
seq_lengths = torch.zeros(batch_size, dtype=torch.long)
for i in range(batch_size):
    for j in range(max_seq - 1, -1, -1):
        if X[i, j] != 0:
            seq_lengths[i] = j + 1
            break

In [59]:
seq_lengths

tensor([ 10,   4,   8,   5,   3,   3,   7,   7,  10,   9,   6,   3,
          3,   6,   3,   3,   8,   8,   9,   4,  10,   7,  10,  10,
          6,   9,   4,  10,   5,   6,  10,   3])

In [32]:
Ys[0][:5]

tensor([ 8,  8,  5,  4,  4])

In [33]:
from metal.end_model import EndModel
from metal.input_modules import LSTMModule



lstm_module = LSTMModule(input_size, hidden_size, vocab_size, bidirectional=bidirectional)
em = EndModel(cardinality=MAX_INT, 
              input_module=lstm_module, 
              layer_output_dims=[hidden_size * (bidirectional + 1), MAX_INT])
em.train(Xs[0], Ys[0], Xs[1], Ys[1], n_epochs=20, print_every=5)
em.score(Xs[2], Ys[2])

Using randomly initialized embeddings.
Embeddings shape = (10, 4)
The embeddings are NOT FROZEN
Using lstm_reduction = 'max'
Overwriting layer_output_dims=[100, 50] to layer_output_dims=[10, 8]

Network architecture:
Sequential(
  (0): Sequential(
    (0): LSTMModule(
      (embeddings): Embedding(10, 4)
      (lstm): LSTM(4, 10, batch_first=True)
    )
    (1): ReLU()
    (2): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): Sequential(
    (0): Linear(in_features=10, out_features=8, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (2): Linear(in_features=8, out_features=8, bias=True)
)

Reaffirming n_epochs=20
Reaffirming print_every=5


RuntimeError: Traceback (most recent call last):
  File "/Users/bradenjh/anaconda3/envs/metal/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/Users/bradenjh/anaconda3/envs/metal/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 138, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/Users/bradenjh/anaconda3/envs/metal/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 138, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/Users/bradenjh/anaconda3/envs/metal/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 115, in default_collate
    return torch.stack(batch, 0, out=out)
RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 3 and 10 in dimension 1 at /Users/soumith/minicondabuild3/conda-bld/pytorch_1524590658547/work/aten